In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
#download the data
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=True
)
test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=ToTensor(),
    download=True
)

100%|████████████████████████████| 9912422/9912422 [00:03<00:00, 3151467.85it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|████████████████████████████████| 28881/28881 [00:00<00:00, 3490942.19it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|████████████████████████████| 1648877/1648877 [00:01<00:00, 1341595.53it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 8048385.62it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [9]:
#create the data loader
from torch.utils.data import DataLoader

loaders = {
    'train':DataLoader(train_data,
                      batch_size=100,
                      shuffle=True,
                      num_workers=1),
    'test':DataLoader(test_data,
                     batch_size=100,
                     shuffle=True,
                     num_workers=1)
}

In [10]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x72721129c4c0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x72721129e0b0>}

In [16]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.softmax(x)

In [17]:
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cuda')

In [18]:
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f"train epoch {epoch} [{batch_idx * len(data)}/{len(loaders['train'].dataset)} ({100. * batch_idx / len(loaders['train']):.0f}%)]")


In [19]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f"\ntest set: Average loss: {test_loss:.4f}, accuracy: {correct}/{len(loaders['test'].dataset)} ({100. * correct / len(loaders['test'].dataset):.2f}%)")


In [20]:
for epoch in range(1,11):
    train(epoch)
    test()

/tmp/ipykernel_7972/2530090726.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


train epoch 1 [0/60000 (0%)]
train epoch 1 [2000/60000 (3%)]
train epoch 1 [4000/60000 (7%)]
train epoch 1 [6000/60000 (10%)]
train epoch 1 [8000/60000 (13%)]
train epoch 1 [10000/60000 (17%)]
train epoch 1 [12000/60000 (20%)]
train epoch 1 [14000/60000 (23%)]
train epoch 1 [16000/60000 (27%)]
train epoch 1 [18000/60000 (30%)]
train epoch 1 [20000/60000 (33%)]
train epoch 1 [22000/60000 (37%)]
train epoch 1 [24000/60000 (40%)]
train epoch 1 [26000/60000 (43%)]
train epoch 1 [28000/60000 (47%)]
train epoch 1 [30000/60000 (50%)]
train epoch 1 [32000/60000 (53%)]
train epoch 1 [34000/60000 (57%)]
train epoch 1 [36000/60000 (60%)]
train epoch 1 [38000/60000 (63%)]
train epoch 1 [40000/60000 (67%)]
train epoch 1 [42000/60000 (70%)]
train epoch 1 [44000/60000 (73%)]
train epoch 1 [46000/60000 (77%)]
train epoch 1 [48000/60000 (80%)]
train epoch 1 [50000/60000 (83%)]
train epoch 1 [52000/60000 (87%)]
train epoch 1 [54000/60000 (90%)]
train epoch 1 [56000/60000 (93%)]
train epoch 1 [58000/6000